<h1>Setting up ther Peritonitis project</h1>

In [1]:
# Connect to database
from cytopy.data.setup import global_init
global_init("Peritonitis")
# Other imports
from cytopy.data.project import Project
from cytopy.data.read_write import get_fcs_file_paths
import pandas as pd

<h2>Create project</h2>

We start by creating a `Project` as we have before.

In [2]:
project = Project(project_id="Immunofingerprints")
project.save()

<Project: Project object>

In [3]:
project = Project.objects(project_id="Immunofingerprints").get()

<h2>Add subjects</h2>

We want to add `Subject`'s to house meta data. The most important variable is going to be the patient status; did they have symptoms of acute peritonitis.

In [4]:
patient_meta = pd.read_csv("patient_categories.csv")

In [5]:
patients = patient_meta.study_id.values

In [6]:
for p in patients:
    meta = patient_meta[patient_meta.study_id == p].to_dict("records")[0]
    project.add_subject(subject_id=p,
                        status=meta.get("status"),
                        infection_data={"bacterial_isolate": meta.get("bacterial_isolate"),
                                        "gram_status": meta.get("gram_status")})

<h2>Create experiment and add files</h2>

We have two experiments for this project, one for the T cell staining the other for general leukocyte staining.

In [7]:
project.add_experiment(experiment_id="Drain fluid T cells", 
                       data_directory="/media/ross/SP PHD U3/PD_Immunofingerprints",
                       panel_definition="panels/T_Panel.xlsx")

<Experiment: Experiment object>

In [8]:
project.add_experiment(experiment_id="Drain fluid Leukocytes", 
                       data_directory="/media/ross/SP PHD U3/PD_Immunofingerprints",
                       panel_definition="panels/L_Panel.xlsx")

<Experiment: Experiment object>

<h3>T cell staining</h3>

This is very similar to what we saw before in the setup of perionitis samples in the validation section.

In [9]:
t_cells = project.get_experiment("Drain fluid T cells")

In [10]:
def filepaths(pt_id, status, panel, ctrls):
    dirpath = f"/media/ross/SP PHD U3/PD_FCS_Data/fcs_data/{status}/{pt_id}/{panel}"
    filepaths = get_fcs_file_paths(dirpath,
                                   control_names=ctrls,
                                   ctrl_id="FMO",
                                   ignore_comp=True,
                                   exclude_dir="DUPLICATE")
    if len(filepaths.get("primary")) == 0:
        print(f"WARNING: no primary files found for {pt_id} in {panel}")
        return None, None
    if len(filepaths.get("primary")) > 1:
        print(f"WARNING: multiple primary files found for {pt_id} in {panel}")
        return None, None
    primary = filepaths.get("primary")[0]
    controls = filepaths.get("controls")
    if len(controls) > 0:
        controls = {x: v[0] for x, v in controls.items()}
    return primary, controls

In [11]:
t_ctrls = ["CXCR3", "CD25", "CD45RA", "CCR7"]

In [12]:
for pt_id, status in patient_meta[["study_id", "status"]].values:
    print(f"======== {pt_id} ========")
    primary, controls = filepaths(pt_id, status, "PDMC_T", t_ctrls)
    if primary is not None:
        t_cells.add_fcs_files(sample_id=pt_id,
                              primary=primary,
                              controls=controls,
                              subject_id=pt_id,
                              compensate=True,
                              verbose=False,
                              missing_error="warn")
    print("==========================\n")

======== 142-09 ========


/home/ross/CytoPy/CytoPy/data/experiment.py:372: UserWarning: Missing channel FSC-W
  warn(f"Missing channel {x.standard}")
/home/ross/CytoPy/CytoPy/data/experiment.py:372: UserWarning: Missing channel SSC-H
  warn(f"Missing channel {x.standard}")



======== 165-09 ========

======== 175-09 ========

======== 209-03 ========

======== 209-05 ========

======== 210-12 ========

======== 210-14 ========

======== 229-02 ========

======== 237-06 ========

======== 239-02 ========

======== 239-04 ========

======== 251-07 ========

======== 251-08 ========

======== 254-04 ========

======== 254-05 ========

======== 255-04 ========

======== 255-05 ========

======== 262-01 ========

======== 264-02 ========

======== 267-01 ========

======== 267-02 ========

======== 272-01 ========

======== 273-01 ========

======== 276-01 ========

======== 279-03 ========

======== 286-02 ========

======== 286-03 ========

======== 286-04 ========

======== 288-02 ========

======== 289-01 ========

======== 294-01 ========

======== 294-02 ========

======== 294-03 ========

======== 295-01 ========

======== 298-01 ========

======== 302-01 ========

======== 305-01 ========

======== 305-02 ========

======== 305-03 ========

======== 30

<h3>Leukocyte staining</h3>

In [13]:
l_cells = project.get_experiment("Drain fluid Leukocytes")

In [14]:
for pt_id, status in patient_meta[["study_id", "status"]].values:
    print(f"======== {pt_id} ========")
    primary, controls = filepaths(pt_id, status, "PDMC_N", [])
    if primary is not None:
        l_cells.add_fcs_files(sample_id=pt_id,
                              primary=primary,
                              subject_id=pt_id,
                              compensate=True,
                              verbose=False,
                              missing_error="warn")
    print("==========================\n")

======== 142-09 ========

======== 165-09 ========

======== 175-09 ========

======== 209-03 ========

======== 209-05 ========

======== 210-12 ========

======== 210-14 ========

======== 229-02 ========

======== 237-06 ========

======== 239-02 ========

======== 239-04 ========

======== 251-07 ========

======== 251-08 ========

======== 254-04 ========

======== 254-05 ========

======== 255-04 ========

======== 255-05 ========

======== 262-01 ========

======== 264-02 ========

======== 267-01 ========

======== 267-02 ========

======== 272-01 ========

======== 273-01 ========

======== 276-01 ========

======== 279-03 ========

======== 286-02 ========

======== 286-03 ========

======== 286-04 ========

======== 288-02 ========

======== 289-01 ========

======== 294-01 ========

======== 294-02 ========

======== 294-03 ========

======== 295-01 ========

======== 298-01 ========

======== 302-01 ========

======== 305-01 ========

======== 305-02 ========

======== 305